In [1]:
import pandas as pd
import numpy as np
import time
import yfinance as yf
from datetime import datetime, timedelta
import pytz
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from concurrent.futures import ThreadPoolExecutor, as_completed
import logging

In [2]:
gpt_path = r"C:\Users\amoog\Desktop\Project_X\Project_X\tweet_buy_or_sell.xlsx"
gpt_df = pd.read_excel(gpt_path)

In [3]:
def return_stock(message):
    if len(message.split()) > 1:
        return message.split()[0].replace('[','').replace(']','').replace('$', '').replace('\\', '').replace('*','').upper()
def return_buy_sell(message):
    if len(message.split()) > 1:
        return 1 if 'Buy' in message.split()[1].replace('[','').replace(']','') else 0
    
gpt_df['Ticker'] = gpt_df['buy_or_sell'].apply(return_stock)
gpt_df['Buy'] = gpt_df['buy_or_sell'].apply(return_buy_sell)

In [4]:
buy_df = gpt_df[(gpt_df['Ticker'].isnull() == False ) & (gpt_df['Buy'] == 1)]
buy_df.head(60)

,id,text,created_at,image_url,Image,jpg_url,image_response,full_response,buy_or_sell,Ticker,Buy
4,1.790077e+18,\\$PARA Open\n\nSee how Sony plays the buyout ...,2024-05-13 17:50:07.000000,['pic.twitter.com/CceV23s8gb'],NaN,https://pbs.twimg.com/media/GNehnPvbMAAq-N7?fo...,The image describes a stock purchase. Specific...,\\$PARA Open\n\nSee how Sony plays the buyout ...,[PARA] [Buy],PARA,1.0
6,1.790015e+18,\\$VKTX Open https://t.co/2G1TFQ8iaV,2024-05-13 13:43:13.000000,['pic.twitter.com/2G1TFQ8iaV'],NaN,https://pbs.twimg.com/media/GNdpGbAbwAA4VSw?fo...,This image describes a stock purchase order fo...,\\$VKTX Open https://t.co/2G1TFQ8iaV TRANSCRI...,[VKTX] [Buy],VKTX,1.0
11,1.789300e+18,The volatility of volatility \\$VVIX is essent...,2024-05-11 14:21:55.000000,['x.com/i/web/status/1…'],NaN,NaN,NaN,The volatility of volatility \\$VVIX is essent...,[$VVIX] [Buy],VVIX,1.0
19,1.788568e+18,\\$SMMT Open\n\nBack in.\n\nI really like trad...,2024-05-09 13:54:43.999999,['pic.twitter.com/WNy2WINVwq'],NaN,https://pbs.twimg.com/media/GNJFYMObkAATrSn?fo...,The image depicts a stock purchase. It shows t...,\\$SMMT Open\n\nBack in.\n\nI really like trad...,[SMMT] [Buy]\n\nBased on the provided text and...,SMMT,1.0
21,1.788565e+18,\\$ULTA Open\n\nAnother LULU type destroyed na...,2024-05-09 13:41:58.000001,['pic.twitter.com/wZl8ISwGxb'],NaN,https://pbs.twimg.com/media/GNJCdJ2aMAEpwVz?fo...,The image describes the purchase of a call opt...,\\$ULTA Open\n\nAnother LULU type destroyed na...,[ULTA] [Buy]\n\nThe message and image data sug...,ULTA,1.0
22,1.788565e+18,\\$BITX Open\n\nSpeculating on a bounce in bit...,2024-05-09 13:41:16.000000,['pic.twitter.com/iigSxnAU8S'],NaN,https://pbs.twimg.com/media/GNJCS_EbcAAUtW-?fo...,The image describes a purchase of a call optio...,\\$BITX Open\n\nSpeculating on a bounce in bit...,[BITX] [Buy]\n\nThe message indicates the purc...,BITX,1.0
25,1.788551e+18,Oh my \\$HOOD going semi nuts in pre market of...,2024-05-09 12:46:24.000000,0,No image available,NaN,NaN,Oh my \\$HOOD going semi nuts in pre market of...,[HOOD] [Buy]\n\nThe message is discussing Robi...,HOOD,1.0
26,1.788550e+18,Today\u2019s watchlist:\n\nLong:\n\\$BITX\n\\$...,2024-05-09 12:42:17.000000,0,No image available,NaN,NaN,Today\u2019s watchlist:\n\nLong:\n\\$BITX\n\\$...,[BITX] [Buy]\n[ULTA] [Buy]\n[SMMT] [Buy],BITX,1.0
28,1.788412e+18,Hang Seng going bonkers overnight.\nIf it stic...,2024-05-09 03:32:37.000000,0,No image available,NaN,NaN,Hang Seng going bonkers overnight.\nIf it stic...,[FXI] [Buy] Hang Seng going bonkers overnight....,FXI,1.0
31,1.788204e+18,\\$FXI Open\n\nWe played the downside in China...,2024-05-08 13:47:11.000000,['pic.twitter.com/aIjR48ncY0'],NaN,https://pbs.twimg.com/media/GND6D5ea8AAZtLf?fo...,This image is describing the purchase of call ...,\\$FXI Open\n\nWe played the downside in China...,[FXI] [Buy]\n\nThe message and transcribed ima...,FXI,1.0


In [9]:
# Define US business day taking into account federal holidays
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def adjust_to_trading_hours(dt):
    """Adjust datetime to the nearest trading hour if outside trading hours, considering weekends and holidays."""
    if dt.weekday() >= 5:  # Saturday (5) or Sunday (6)
        dt = dt + us_bd  # Move to next business day
        logging.info(f"Adjusted for weekend to: {dt}")
    if dt.hour < 9 or (dt.hour == 9 and dt.minute < 30):
        dt = dt.replace(hour=9, minute=30, second=0, microsecond=0)
        logging.info(f"Adjusted for early hour to: {dt}")
    elif dt.hour > 16:
        dt = (dt + us_bd).replace(hour=9, minute=30, second=0, microsecond=0)
        logging.info(f"Adjusted for late hour to next business day: {dt}")
    return dt

def interval_to_minutes(interval):
    """Convert a yfinance interval string to minutes."""
    unit = interval[-1]
    if unit == 'm':
        return int(interval[:-1])
    elif unit == 'h':
        return int(interval[:-1]) * 60
    elif unit == 'd':
        return int(interval[:-1]) * 390  # Assuming 6.5 trading hours per day
    elif unit == 'w':
        return int(interval[:-1]) * 1950  # Assuming 5 trading days per week
    elif unit == 'mo':
        return int(interval[:-1]) * 8190  # Rough estimate for a month
    return None

def safe_pct_change(data, callout_price, closest_time_index, index_offset):
    """Calculate safe percent change considering bounds."""
    target_index = closest_time_index + index_offset
    if target_index < len(data):
        return (data['Close'].iloc[target_index] - callout_price) / callout_price * 100
    return (data['Close'].iloc[-1] - callout_price) / callout_price * 100  # Return until the last available data

def calculate_volatility(data, days=30):
    """ Calculate the standard deviation of daily returns for the given number of days. """
    daily_returns = data['Close'].pct_change().dropna()
    return daily_returns.tail(days).std()
def calculate_profitability_score(row, max_profit_global):
    base_score = 0
    total_possible_score = 0
    profit_weight = 10
    time_penalty_weight = 5

    for i in range(1, 4):
        profit_key = f'Tertile {i} Profit %'
        time_key = f'Tertile {i} Time to Target Hours'
        drop_key = f'Tertile {i} Drop Before Rise %'

        profit = row.get(profit_key, 0)
        time_to_target = row.get(time_key, float('inf'))
        drop_before_rise = row.get(drop_key, 0)

        if profit is not None:
            possible_score = max_profit_global * profit_weight
            total_possible_score += possible_score
            drop_penalty = abs(drop_before_rise) * 2 if drop_before_rise < 0 else 0  # Apply drop penalty only for negative drops
            time_penalty = (time_to_target / 24) * time_penalty_weight
            base_score += profit * profit_weight - time_penalty - drop_penalty

        logging.info(f"Tertile {i}: Profit={profit}, Time Penalty={time_penalty}, Drop Penalty={drop_penalty}")

    normalized_score = (base_score / total_possible_score) * 100 if total_possible_score > 0 else 0
    logging.info(f"Normalized Profitability Score: {normalized_score}")
    return normalized_score

def calculate_risk_score(row, data):
    risk_weight = 2
    time_penalty = 0.1
    drop_penalty = 3  # Increased drop penalty to reflect higher risk for volatile drops

    volatility = calculate_volatility(data)
    adjusted_risk_weight = risk_weight * (1 + volatility)

    max_loss = abs(row['Max_Loss'])
    time_to_max_loss = row.get('Time to Max Loss', 0) / 24

    # Consider drops from all tertiles in the risk calculation, only apply penalties for negative values
    total_drop_penalty = sum(abs(row.get(f'Tertile {i} Drop Before Rise %', 0)) * drop_penalty if row.get(f'Tertile {i} Drop Before Rise %', 0) < 0 else 0 for i in range(1, 4))

    risk_score = max_loss * adjusted_risk_weight + time_to_max_loss * time_penalty + total_drop_penalty
    return risk_score


def fetch_and_analyze_performance(ticker, signal_date):
    local_tz = pytz.timezone('America/New_York')  # Assuming NYSE timezone
    signal_date = adjust_to_trading_hours(signal_date)
    signal_date_utc = signal_date.astimezone(pytz.utc)  # Convert signal date to UTC for querying
    end_date_utc = signal_date_utc + timedelta(days=7)

    attempts = 20
    wait = 0.001  # initial wait time in seconds
    interval = '1m'  # smaller interval for more granular data
    
    intervals_per_hour = 60 / interval_to_minutes(interval)
    intervals_per_day = 390 / interval_to_minutes(interval)
    intervals_per_week = 1950 / interval_to_minutes(interval)
    
    for attempt in range(attempts):
        try:
            data = yf.download(ticker, start=signal_date_utc.strftime('%Y-%m-%d'), end=end_date_utc.strftime('%Y-%m-%d'), interval=interval, progress=False)
            if data.empty:
                logging.warning(f"No data found for {ticker} on attempt {attempt + 1}. Skipping...")
                return None

            data.index = data.index.tz_convert(local_tz)  # Convert data index to local time zone
            logging.info(f"Successfully downloaded data for {ticker}")

            closest_time_index = data.index.get_loc(signal_date, method='nearest')
            callout_price = data.iloc[closest_time_index]['Close']
            max_price = data['High'].max()  # Maximum high price in the period
            max_price_index = data['High'].idxmax()
            min_price_index = data['Low'].idxmin()
            max_gain = ((max_price - callout_price) / callout_price) * 100
            max_loss = ((data['Low'].min() - callout_price) / callout_price) * 100

            time_to_max_gain = (max_price_index - data.index[closest_time_index]).total_seconds() / 3600 if max_price_index >= data.index[closest_time_index] else None
            time_to_max_loss = (min_price_index - data.index[closest_time_index]).total_seconds() / 3600 if min_price_index >= data.index[closest_time_index] else None

            logging.info(f"Using callout price from index {closest_time_index}, price: {callout_price}")
            logging.info(f"Max price at {max_price_index}, min price at {min_price_index}")
            logging.info(f"Calculated max gain: {max_gain}%, max loss: {max_loss}%")
            if time_to_max_gain is not None:
                logging.info(f"Time to max gain: {time_to_max_gain} hours after callout.")
            else:
                logging.info("Max gain occurred before the callout time.")
            if time_to_max_loss is not None:
                logging.info(f"Time to max loss: {time_to_max_loss} hours after callout.")
            else:
                logging.info("Max loss occurred before the callout time.")
                
            results = {
                'Ticker': ticker,
                'Return_1hr': safe_pct_change(data, callout_price, closest_time_index, int(intervals_per_hour)),
                'Return_1d': safe_pct_change(data, callout_price, closest_time_index, int(intervals_per_day)),
                'Return_1wk': safe_pct_change(data, callout_price, closest_time_index, int(intervals_per_week)),
                'Max_Gain': max_gain,
                'Max_Loss': max_loss
            }
            tertile_size = (max_price - callout_price) / 3
            tertile_targets = [callout_price + tertile_size * i for i in range(1, 4)]
            for i, target_price in enumerate(tertile_targets, start=1):
                above_target = data[data['High'] >= target_price]
                if not above_target.empty:
                    first_reach_idx = above_target.index[0]
                    pre_target_data = data[:first_reach_idx]
                    min_before_target = pre_target_data['Low'].min()
                    drop_before_rise = (min_before_target - callout_price) / callout_price * 100
                    time_to_target = max(0, (first_reach_idx - data.index[closest_time_index]).total_seconds() / 3600)  # Ensure non-negative

                    results[f'Tertile {i} Profit %'] = ((target_price - callout_price) / callout_price) * 100
                    # Adjusting risk level based on drop percentage and time to target
                    if drop_before_rise < 0:
                        results[f'Tertile {i} Risk Level'] = 'low' if time_to_target < 1 else 'medium' if time_to_target < 5 else 'high'
                    else:
                        results[f'Tertile {i} Risk Level'] = 'low'
                    results[f'Tertile {i} Drop Before Rise %'] = drop_before_rise
                    results[f'Tertile {i} Time to Target Hours'] = time_to_target
                else:
                    results[f'Tertile {i} Profit %'] = None
                    results[f'Tertile {i} Risk Level'] = 'No target reached'
                    results[f'Tertile {i} Drop Before Rise %'] = None
                    results[f'Tertile {i} Time to Target Hours'] = None
            max_profit_global = max(results[key] for key in ['Tertile 1 Profit %', 'Tertile 2 Profit %', 'Tertile 3 Profit %'])
            profitability_score = calculate_profitability_score(results, max_profit_global)
            risk_score = calculate_risk_score(results, data)
            results['Profitability Score'] = profitability_score
            results['Risk Score'] = risk_score
            return results
        except Exception as e:
            time.sleep(wait)
            wait *= 2  # exponential backoff
            if attempt == attempts - 1:
                logging.error(f"Failed to download data for {ticker} on attempt {attempt + 1}: {e}")
    return None

def parallel_download(df):
    with ThreadPoolExecutor(max_workers=2) as executor:
        futures = {executor.submit(fetch_and_analyze_performance, row['Ticker'], pd.to_datetime(row['created_at']).tz_localize(pytz.timezone('America/New_York'))): row for index, row in df.iterrows()}
        results = [future.result() for future in as_completed(futures) if future.result() is not None]
    return pd.DataFrame(results)

# Assuming result_df is the dataframe you have:
start_time = datetime.now()
result_df = parallel_download(df=buy_df)
# Applying the scoring function
end_time = datetime.now()
logging.info(f"Time taken with parallel processing: {(end_time - start_time).total_seconds():.2f} seconds")


2024-05-17 20:11:20,102 - INFO - Adjusted for late hour to next business day: 2024-05-14 09:30:00-04:00
2024-05-17 20:11:20,340 - INFO - Successfully downloaded data for VKTX
2024-05-17 20:11:20,342 - INFO - Successfully downloaded data for PARA
2024-05-17 20:11:20,344 - INFO - Using callout price from index 252, price: 71.04000091552734
2024-05-17 20:11:20,348 - INFO - Using callout price from index 0, price: 13.210000038146973
2024-05-17 20:11:20,348 - INFO - Max price at 2024-05-15 13:17:00-04:00, min price at 2024-05-17 15:54:00-04:00
2024-05-17 20:11:20,349 - INFO - Max price at 2024-05-14 09:32:00-04:00, min price at 2024-05-14 14:09:00-04:00
2024-05-17 20:11:20,350 - INFO - Calculated max gain: 12.443688378329469%, max loss: -6.137388167457905%
2024-05-17 20:11:20,351 - INFO - Calculated max gain: 1.1294434924145351%, max loss: -9.992427638246859%
2024-05-17 20:11:20,352 - INFO - Time to max gain: 47.56666666666667 hours after callout.
2024-05-17 20:11:20,354 - INFO - Time to ma


1 Failed download:
1 Failed download:
- VVIX: No data found, symbol may be delisted

- VVIX: No data found, symbol may be delisted


2024-05-17 20:11:20,732 - INFO - Successfully downloaded data for ULTA
2024-05-17 20:11:20,734 - INFO - Using callout price from index 167, price: 394.2200012207031
2024-05-17 20:11:20,735 - INFO - Max price at 2024-05-13 09:49:00-04:00, min price at 2024-05-09 09:30:00-04:00
2024-05-17 20:11:20,735 - INFO - Calculated max gain: 4.408707023741889%, max loss: -1.5777995031816947%
2024-05-17 20:11:20,736 - INFO - Time to max gain: 92.11666666666666 hours after callout.
2024-05-17 20:11:20,736 - INFO - Max loss occurred before the callout time.
2024-05-17 20:11:20,749 - INFO - Tertile 1: Profit=1.4695690079139632, Time Penalty=4.222222222222222, Drop Penalty=3.1555990063633894
2024-05-17 20:11:20,755 - INFO - Tertile 2: Profit=2.9391380158279263, Time Penalty=19.131944444444443, Drop Penalty=3.1555990063633894
2024-05-17 20:11:20,756 - INFO - Tertile 3: Profit=4.408707023741889, Time Penalty=19.19097222222222, Drop Penalty=3.1555990063633894
2024-05-17 20:11:20,757 - INFO - Normalized Pro

2024-05-17 20:11:21,900 - INFO - Tertile 1: Profit=7.412113619006669, Time Penalty=4.5, Drop Penalty=9.911051388989703
2024-05-17 20:11:21,901 - INFO - Time to max loss: 1.0666666666666667 hours after callout.
2024-05-17 20:11:21,902 - INFO - Tertile 2: Profit=14.824227238013313, Time Penalty=24.15625, Drop Penalty=9.911051388989703
2024-05-17 20:11:21,903 - INFO - Tertile 3: Profit=22.236340857019982, Time Penalty=29.125, Drop Penalty=9.911051388989703
2024-05-17 20:11:21,905 - INFO - Normalized Profitability Score: 53.54784093754632
2024-05-17 20:11:21,906 - INFO - Tertile 1: Profit=7.77777628341635, Time Penalty=5.375, Drop Penalty=8.205132406060331
2024-05-17 20:11:21,913 - INFO - Tertile 2: Profit=15.5555525668327, Time Penalty=25.041666666666668, Drop Penalty=8.205132406060331
2024-05-17 20:11:21,914 - INFO - Tertile 3: Profit=23.33332885024905, Time Penalty=30.01041666666667, Drop Penalty=8.205132406060331
2024-05-17 20:11:21,914 - INFO - Normalized Profitability Score: 54.51773

2024-05-17 20:11:23,321 - INFO - Calculated max gain: 12.454522306050187%, max loss: -9.207951182977887%
2024-05-17 20:11:23,336 - INFO - Time to max gain: 266.3833333333333 hours after callout.
2024-05-17 20:11:23,337 - INFO - Time to max loss: 98.36666666666666 hours after callout.
2024-05-17 20:11:23,356 - INFO - Tertile 1: Profit=4.151507435350062, Time Penalty=14.142361111111112, Drop Penalty=5.541549826214006
2024-05-17 20:11:23,367 - INFO - Tertile 2: Profit=8.303014870700125, Time Penalty=14.142361111111112, Drop Penalty=5.541549826214006
2024-05-17 20:11:23,422 - INFO - Tertile 3: Profit=12.454522306050187, Time Penalty=55.49652777777777, Drop Penalty=18.415902365955773
2024-05-17 20:11:23,434 - INFO - Normalized Profitability Score: 36.34829469042094
2024-05-17 20:11:23,437 - INFO - Successfully downloaded data for FXI
2024-05-17 20:11:23,439 - INFO - Adjusted for weekend to: 2024-05-06 19:08:31-04:00
2024-05-17 20:11:23,452 - INFO - Using callout price from index 239, price:

In [10]:
result_df

,Ticker,Return_1hr,Return_1d,Return_1wk,Max_Gain,Max_Loss,Tertile 1 Profit %,Tertile 1 Risk Level,Tertile 1 Drop Before Rise %,Tertile 1 Time to Target Hours,Tertile 2 Profit %,Tertile 2 Risk Level,Tertile 2 Drop Before Rise %,Tertile 2 Time to Target Hours,Tertile 3 Profit %,Tertile 3 Risk Level,Tertile 3 Drop Before Rise %,Tertile 3 Time to Target Hours,Profitability Score,Risk Score
0,VKTX,1.041664,1.745492,-5.686938,12.443688,-6.137388,4.147896,low,4.377816,0.000000,8.295792,high,-0.858672,47.066667,12.443688,high,-0.858672,47.566667,60.465406,17.439832
1,PARA,0.567750,-5.373958,-9.008324,1.129443,-9.992428,0.376481,low,-0.340652,0.016667,0.752962,low,-0.340652,0.033333,1.129443,low,-0.340652,0.033333,60.583226,23.070138
2,SMMT,0.625646,-1.981236,-7.403546,7.574551,-7.924924,2.524850,low,4.483840,0.000000,5.049700,low,4.483840,0.000000,7.574551,low,4.483840,0.000000,66.666667,15.889974
3,ULTA,0.238446,1.859364,1.354573,4.408707,-1.577800,1.469569,high,-1.577800,20.266667,2.939138,high,-1.577800,91.833333,4.408707,high,-1.577800,92.116667,27.341504,17.361746
4,BITX,-0.052412,-5.372115,15.775683,15.775683,-6.551103,5.258561,high,-6.551103,139.816667,10.517122,high,-6.551103,144.800000,15.775683,high,-6.551103,194.316667,37.278737,72.136408
5,BITX,-1.370214,-5.687696,14.219235,14.219235,-7.807397,4.739745,high,-7.807397,140.800000,9.479490,high,-7.807397,145.966667,14.219235,high,-7.807397,195.300000,32.141850,85.969891
6,HOOD,-0.979545,-3.860559,15.759151,15.759151,-6.539904,5.253050,low,8.959956,0.000000,10.506101,low,8.959956,0.000000,15.759151,high,-6.539904,195.233333,55.296888,32.902689
7,FXI,-0.437952,2.006935,7.043797,7.043797,-0.839414,2.347932,high,-0.839414,96.000000,4.695865,high,-0.839414,198.500000,7.043797,high,-0.839414,198.500000,15.678626,9.244428
8,FXI,-0.437952,2.006935,7.043797,7.043797,-0.839414,2.347932,high,-0.839414,96.000000,4.695865,high,-0.839414,198.500000,7.043797,high,-0.839414,198.500000,15.678626,9.244428
9,INTC,0.470197,1.035432,6.901764,6.901764,-0.151134,2.300588,high,-0.151134,115.883333,4.601176,high,-0.151134,218.266667,6.901764,high,-0.151134,218.266667,10.645432,1.663885


In [7]:

# Tertile 1 Drop before rise is positive, what does it mean? Why is it often bigger than Tertile1%? Does this make sense? Please fix it if not.
# HOOD Risk level seems to be backwards, since it is high for something that seems to profit in no time? Does it really do this?

In [8]:
print(end_time-start_time)

0:00:04.452044
